In [10]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

# Adjust Pandas display settings
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')

# Inner  Join 


Datasets for example

The city of Chicago is ivided into fifty local neighbordooids called wards. We have table with data about the local goernment offices in each ward. In this example, we want to merge the local government data with census data about the popultaion of each ward. 



### The ward data

In [11]:
# Load the .p file 
ward_data = pd.read_pickle("/home/khaijones/Data-Camp/Merging with Pandas Data Sets/ward.p")

# Print the DataFrame
print(ward_data.head())
print(ward_data.shape)

  ward       alderman                  address               zip  
0   1   Proco "Joe" Moreno        2058 NORTH WESTERN AVENUE  60647
1   2        Brian Hopkins       1400 NORTH  ASHLAND AVENUE  60622
2   3           Pat Dowell          5046 SOUTH STATE STREET  60609
3   4     William D. Burns  435 EAST 35TH STREET, 1ST FLOOR  60616
4   5   Leslie A. Hairston            2325 EAST 71ST STREET  60649
(50, 4)


This table has 3519 rows and 5 columns